In [4]:
import socket

HOST = '0.0.0.0'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

#HOST 10.17.35.207

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)

OSError: [Errno 98] Address already in use

In [4]:
import socket



with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)

print('Received', repr(data))

Received b'hi\r\n'


In [5]:
import select, socket, sys, queue
import logging

#initialization of logging
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.DEBUG,datefmt="%H:%M:%S")

#initialistation of sockets
HOST = '0.0.0.0'  # The server's hostname or IP address
PORT = 65432        # The port used by the server

test_var = None

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server:
    server.setblocking(0)
    server.bind((HOST, PORT))
    server.listen()#.listen(5)
    inputs = [server]
    outputs = []
    message_queues = {}
    
    logging.debug("Server initialised")
    
    while True:
        readable, writable, exceptional = select.select(
            inputs, outputs, inputs,5)
        for s in readable:
            if s is server:
                connection, client_address = s.accept()
                connection.setblocking(0)
                inputs.append(connection)
                message_queues[connection] = queue.Queue()
                test_var = connection
                logging.debug("Server: got new +++ connection +++ name: %s, address: %s",connection,client_address)
            else:
                data = s.recv(1024)
                if data:
                    logging.debug("Server: message from: %s, message: %s",s,data)
                    message_queues[s].put(data)
                    if s not in outputs:
                        outputs.append(s)
                else:
                    logging.debug("Server: message from: %s, empty message, XXXX closing connection XXX",s)
                    if s in outputs:
                        outputs.remove(s)
                    inputs.remove(s)
                    s.close()
                    del message_queues[s]

        for s in writable:
            try:
                next_msg = message_queues[s].get_nowait()
            except queue.Empty:
                outputs.remove(s)
            else:
                logging.debug("Server: message send back to: %s, message: %s",s,next_msg)
                s.send(next_msg)

        for s in exceptional:
            logging.debug("Server: error communication with: %s, closing connection",s)
            inputs.remove(s)
            if s in outputs:
                outputs.remove(s)
            s.close()
            del message_queues[s]
        logging.debug("Cycle")

01:50:10: Server initialised
01:50:15: Cycle


KeyboardInterrupt: 

In [26]:
help(test_var)

Help on socket in module socket object:

class socket(_socket.socket)
 |  A subclass of _socket.socket adding the makefile() method.
 |  
 |  Method resolution order:
 |      socket
 |      _socket.socket
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, proto=0, fileno=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Wrap __repr__() to reveal the real class name and socket
 |      address(es).
 |  
 |  accept(self)
 |      accept() -> (socket object, address info)
 |      
 |      Wait for an incoming connection.  Return a new socket
 |      representing the connection, and the address of the client.
 |      For IP sockets, the address info is a pair (hostaddr, port).
 |  
 |  close(self)
 |      close()
 |      
 |      Close the socket.  It cann

In [41]:
print('.'.join(map(str,test_var.getpeername())))

10.17.35.137.64762


In [50]:
partner_dict={}
partner_name = "10.17.35.137.64762"
partner_conn = test_var
partner_dict[partner_name]=partner_conn
partner_dict

{'10.17.35.137.64762': <socket.socket fd=60, family=AddressFamily.AF_INET, type=2049, proto=0, laddr=('172.17.0.2', 65432), raddr=('10.17.35.137', 64762)>}

In [37]:
import select
import sys
import os

loop = True




p = os.popen('ls -la')
# добавляем в список стандартный ввод
# и поток с чтением результата запущенного процесса
inputt = [p]

while loop:
    r,w,e = select.select(inputt, [], []) # функция опроса устройств
    for op in r:
        if op == sys.stdin:
            print (':',op.readline().rstrip())
        else:
            s = op.readline()
            # получать больше нечего, удаляем из списка опрашиваемых
            if not s: inputt.remove(op)
            else:
                print ('>',s.rstrip())
            

> total 652
> drwxrwxrwx 1 1000 staff   4096 Aug  8 17:09 .
> drwxrwxrwx 1 1000 staff  12288 Aug  8 13:28 ..
> drwxrwxrwx 1 1000 staff   4096 Aug  7 20:36 .git
> -rwxrwxrwx 1 1000 staff   1307 Jul 31 21:16 .gitignore
> drwxrwxrwx 1 1000 staff   4096 Aug  7 18:48 .ipynb_checkpoints
> -rwxrwxrwx 1 1000 staff  11487 Aug  2 21:07 Preparation1_classes-Copy1.ipynb
> -rwxrwxrwx 1 1000 staff  24715 Aug  7 18:46 Preparation1_classes.ipynb
> -rwxrwxrwx 1 1000 staff 523300 Aug  8 17:09 Preparation2_socket_server.ipynb
> -rwxrwxrwx 1 1000 staff  10026 Aug  7 17:17 test.py
> -rwxrwxrwx 1 1000 staff  61039 Aug  7 17:29 testing_notebook.ipynb


KeyboardInterrupt: 